In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time

In [39]:
# load dataset and labels
X_train = pickle.load(open('pickle_data/X_train.pickle','rb'))
y_train = pickle.load(open('pickle_data/y_train.pickle','rb'))

# load validation dataset and labels
X_validation = pickle.load(open('pickle_data/X_validation.pickle', 'rb'))
y_validation = pickle.load(open('pickle_data/y_validation.pickle', 'rb'))

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_validation.shape =', X_validation.shape)
print('y_validation.shape =', y_validation.shape)

X_train.shape = (2320, 50, 50, 3)
y_train.shape = (2320,)
X_validation.shape = (258, 50, 50, 3)
y_validation.shape = (258,)


In [40]:
# normalize pixel data
X_train = X_train/255.0
X_validation = X_validation/255.0

In [41]:
# define hyperparameters
conv_layer = 4
layer_size = 128
dense_layer = 0

In [42]:
# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
# create a new log of the model to analyze results
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# define model
model = Sequential()

# 1st convolution
model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional conv layers
for _ in range(conv_layer-1):
    # convolution
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # Max-Pool to reduce tensor size while retaining distict features
    model.add(MaxPooling2D(pool_size=(2,2)))

# flatten model for fully connected layers
model.add(Flatten())

# add dense layers
for _ in range(dense_layer):
    # fully-connected
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# final fully-connected layer (5 matches number of Pokemon categories)
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])


Train on 1856 samples, validate on 464 samples
Epoch 1/20
1856/1856 [==============================] - 12s 6ms/sample - loss: 0.8399 - accuracy: 0.7144 - val_loss: 1.2004 - val_accuracy: 0.5323
Epoch 2/20
1856/1856 [==============================] - 11s 6ms/sample - loss: 0.4750 - accuracy: 0.8324 - val_loss: 0.9531 - val_accuracy: 0.6681
Epoch 3/20
1856/1856 [==============================] - 12s 6ms/sample - loss: 0.4200 - accuracy: 0.8508 - val_loss: 1.1644 - val_accuracy: 0.5453
Epoch 4/20
1856/1856 [==============================] - 11s 6ms/sample - loss: 0.3299 - accuracy: 0.8874 - val_loss: 0.4309 - val_accuracy: 0.8750
Epoch 5/20
1856/1856 [==============================] - 11s 6ms/sample - loss: 0.2619 - accuracy: 0.9111 - val_loss: 0.7193 - val_accuracy: 0.7974
Epoch 6/20
1856/1856 [==============================] - 11s 6ms/sample - loss: 0.2278 - accuracy: 0.9203 - val_loss: 0.5062 - val_accuracy: 0.8082
Epoch 7/20
1856/1856 [==============================] - 11s 6ms/sample 

In [43]:
# evaluate the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 89.92%


In [13]:
# save the model
model.save('models/' + NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/2-conv-128-nodes-1-dense-1574830346/assets
